## Tensor operation

We can build a network by stcaking `Dense` layers on top of each other:

`keras.layers.Dense(512, activation='relu')` <=> `output = relu(dot(W, input) + b)`

We have three tensor operations here: 
* a dot product `( dot )` between the input tensor and a tensor named `W`
* an addition `( + )` between the resulting 2D tensor and `a` vector `b` 
* finally, a `relu` operation. `relu(x)` is `max(x, 0)`

### Element-wise operation

* Operations that are applied indpendently to each element in the tensor.
* These operations are highly open to _parallel_ implementation or _vectorized_ implementation.

In [1]:
import numpy as np

# naive parallel implementation of addition for 2D tensor
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    # avoid overwriting input tensor
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

x = np.random.random([2,2])
y = np.random.random([2,2])
np.array_equal(naive_add(x, y), (x + y))

True

### Broadcasting 

When two tensors of different lenght are done some operation to together then the smaller tensor is _broadcasted_ to match the shape of the larger tensor. Steps in brpadcasting:

1. Axes (called _broadcast axes_) are added to the smaller tensor to match the `ndim` of the larger tensor
2. The smaller tensor is repeated alongside these new axes to match the full shape of the larger tensor.

Let’s look at a concrete example. Consider X with shape `(32, 10)` and y with shape
`(10,)` . First, we add an empty first axis to y , whose shape becomes (1, 10) . Then, we
repeat y 32 times alongside this new axis, so that we end up with a tensor Y with shape
`(32, 10)` , where `Y[i, :] == y for i in range(0, 32)`.

**NOTE**: In terms of implementation, no new 2D tensor is created, because that would be
terribly inefficient. The repetition operation is entirely virtual: it happens at the algo-
rithmic level rather than at the memory level.

In [2]:
# navie implementation of broadcasting
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

x = np.random.random([2,2])
y = np.random.random([2])
print(x, '\n\t  +\n', y, '\n\t  =\n', naive_add_matrix_and_vector(x, y))

[[0.17580333 0.14926912]
 [0.36112706 0.29482549]] 
	  +
 [0.41058159 0.16691853] 
	  =
 [[0.58638493 0.31618765]
 [0.77170866 0.46174401]]


### Tensor dot

The dot operation combines entries in the input tensor which is not equal to the _element-wise multiplication_. Mathematically we write `z = x . y`.

In [8]:
x = np.random.random(2)
y = np.random.random(2)
print("\n".join([str(x), str(y), str(np.dot(x, y))]))

[0.57124541 0.83774684]
[0.56824558 0.79969029]
0.9945456900702146


In [12]:
# naive implementation of dot product
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z=0
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

np.array_equal(naive_vector_dot(x, y), np.dot(x, y))

True

In [31]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

mat = np.random.random([2,2])
np.array_equal(naive_matrix_vector_dot(mat, y), np.dot(mat, y))

True

### Tensor Reshaping

* Rehaping a tensor means rearranging its tows and columns to match the target shape. 
* The reshaped tensor has same total number of coefficents as the initial tensor.

In [38]:
x = np.array([[0, 1],
             [2, 3],
             [4, 5]])
print("Before Reshaping:", x.shape)

x = x.reshape((6, 1))
print("\nAfter Reshping:", x.shape)

Before Reshaping: (3, 2)

After Reshping: (6, 1)


A special case of reshaping that's commonly encountered is _transposition_. _Transposing_ a matrix means exchanging its rows and its columns, so that `x[i, :]` becomes `x[:, i]`.